In [0]:
%tensorflow_version 1.x

In [0]:
### Do just once In google Colab left pane: ###
# Upload into Files:
  # neural.zip
  # data.zip
  # helpers_models.py
  # helpers_strings.py

In [0]:
### run just once: ### -- downloads tardan 15 min aprox
# !unzip data.zip
# !wget http://nlp.stanford.edu/data/glove.6B.zip
# !wget http://nlp.stanford.edu/data/glove.twitter.27B.zip

In [0]:
!unzip -n glove.6B.zip
!unzip -n glove.twitter.27B.zip
!unzip -o neural.zip

In [0]:
import pandas as pd
import numpy as np

import helpers_models as hm

from neural.architectures import build_gru
from neural.core.training import full_nn

In [0]:
RUN_SANITY = False
NEW_RANDOM_SPLIT = True
LOAD_PRETRAINED_VECS = True # False no genera el dict de embeddings (xq tarda unos min)

In [0]:
if NEW_RANDOM_SPLIT:
  semilla = np.random.randint(1,9999)
else:
  semilla = 800
rng = np.random.RandomState(semilla)
print(semilla)

In [0]:
datos_raw = hm.read_tagged_data()
datos = hm.clean_tagged_data(datos_raw)
X = datos['text']
y = datos['target']
X_train, X_val, y_train, y_val = hm.split_data(X, y, 0.2, rng)

In [0]:
if LOAD_PRETRAINED_VECS:
  # load pretrained embeddings glove_twitter
  embeddings_twitter = {}
  with open('glove.twitter.27B.200d.txt') as f:
    for line in f:
      word, coefs = line.split(maxsplit=1)
      coefs = np.fromstring(coefs, 'f', sep=' ')
      embeddings_twitter[word] = coefs
  # load pretrained embeddings glove_wiki
  embeddings_wiki = {}
  with open('glove.6B.300d.txt') as f:
    for line in f:
      word, coefs = line.split(maxsplit=1)
      coefs = np.fromstring(coefs, 'f', sep=' ')
      embeddings_wiki[word] = coefs

In [0]:
### GRU sanity check ###
if RUN_SANITY:
  param_tokenizer = dict(
                      vocab_size=None
                      ,pad_type='pre'
                      ,seq_maxlen=100
                      )
  param_nn = dict(
                        optimizer='adam'
                        ,learn_rate=1e-5
                        ,l2_strength=0.0
                        ,decay_strength=0.0
                        ,momentum=None          
                        ,embeddings=embeddings_twitter
                        ,initializer='he_uniform'  
                  )
  param_train = dict(
                        model_id='sanity'
                        ,epochs=20
                        ,batch_size=32
                        ,early_stopping_n=999999
                        ,decay_factor=0.0
                        ,decay_patience_n=999999
                        ,sanity_check_n=20
                        ,verbose=0
                    )
  param_sanity = dict(**param_tokenizer, **param_nn, **param_train)
  mod_check = full_nn(
                build_gru, X_train, y_train, X_val, y_val, **param_sanity
                ) 

In [0]:
# NN parameters
param_tokenizer = dict(
                      vocab_size=None
                      ,pad_type='pre'
                      ,seq_maxlen=100
                      )
param_nn = dict(
                      optimizer='adam'
                      ,learn_rate=1e-5
                      ,l2_strength=0.01
                      ,decay_strength=0.0
                      ,momentum=None          
                      ,embeddings=embeddings_wiki
                      ,initializer='he_uniform'
                )
param_train = dict(
                      model_id='GloveWiki01'
                      ,epochs=200
                      ,batch_size=16
                      ,early_stopping_n=20
                      ,decay_factor=0.8
                      ,decay_patience_n=10
                      ,sanity_check_n=None
                      ,verbose=2
                  )
param = dict(**param_tokenizer, **param_nn, **param_train)

# Training NN
mod = full_nn(build_gru, X_train, y_train, X_val, y_val, **param)

In [0]:
# Agregar RandonSearch (o bayesian para los aventureros)